<a href="https://colab.research.google.com/github/niveamp/modelo-preditivo-titulo-tesouro/blob/main/arrecada_novo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='blue'>TCC BIG DATA</font>
# <font color='blue'>Análise de série temporal de vendas do tesouro e arrecadação federal</font>

## Download: http://github.com/dsacademybr

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

### DADOS ABERTOS DA ARRECADAÇÃO DISPONÍVEL EM:
https://receita.economia.gov.br/dados/receitadata/arrecadacao/analise-gerencial-da-arrecadacao-angela-1/angela-arrecadacao-por-mes-cnae-e-tributo.xlsx/view e 
### DADOS ABERTOS DO TESOURO DIRETO DISPONÍVEL EM:
https://www.tesourotransparente.gov.br/ckan/dataset/f0468ecc-ae97-4287-89c2-6d8139fb4343/resource/e5f90e3a-8f8d-4895-9c56-4bb2f7877920/download/VendasTesouroDireto.csv

### Primeiro dataset com Dados histórico da arrecadação desde 2016 até 2020, por CNAE, natureza jurídica e tributo
### Segundo dataset com dados históricos de vendas do tesouro por título desde 2002 até 2020

In [ ]:
#PARA GOOGLE COLAB
!pip install pmdarima
!pip install keras
!pip install fbprophet
!apt-get -qq install -y graphviz && pip install pydot
!pip install html5lib

In [ ]:
%matplotlib inline
# Importando os pacotes 
import numpy as np
import pandas as pd
import matplotlib as mat
import matplotlib.pyplot as plt
import seaborn as sns
import colorsys
plt.style.use('seaborn') 

import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

#import datetime
import math

#Bibliotecas para utilização das métricas do Sklearn
import sklearn.metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


# Bibliotecas para uso do statsmodels
import statsmodels.api as sm
from scipy import stats

#Bibliotecas para utilização do Keras
from keras.models import Sequential
from keras.utils import plot_model
from keras.layers import LSTM, SimpleRNN, Dense, Dropout, Masking, Embedding


#Bibliotecas para utilização do ARIMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from pandas.plotting import lag_plot

#Pacotes que exigiram instalação no anaconda com pip install
#import pandas_profiling as pdp
import pmdarima as pm


#Bibliotecas para utilização do Prophet

# tive que instalar com conda install -c conda-forge fbprophet
from fbprophet import Prophet
from fbprophet.diagnostics import cross_validation, performance_metrics
from fbprophet.plot import plot_cross_validation_metric


In [ ]:
#np.__version__
#mat.__version__
#pd.__version__

# Etapa de processamento e tratamento dos dados
Essa etapa foi revista após análise e exploração dos dados onde foram identificadas necessidade de alterar colunas, campos,...

In [ ]:
#Carregando arquivo com dados históricos da arrecadação por mês/ano/cnae
arrec_cnae = "arr_cnae_tributo.xlsx"

#Carregando dados históricos de vendas do tesouro direto
vendas_tesouro = "VendasTesouroDireto.csv"



In [ ]:
#Carregando o arquivo .xsls no dataset
dfc = pd.read_excel(arrec_cnae, dtype = None)

In [ ]:
# Carregando o arquivo .csv
dft = pd.read_csv(vendas_tesouro, sep = ';', dtype=None, low_memory=False)

In [ ]:
#Carregando dados históricos Pib
#lpib= pd.read_html('http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=521274780&module=M')

# Análise Exploratória de Dados


In [ ]:
#Visualização do dataset
dfc

In [ ]:
#Identificando campos nulos e tipos de dados. 
dfc.info()

In [ ]:
#Substitui string dos meses por numeração sequencial
meses = {'Jan': 1, 'Fev': 2,'Mar':3, 'Abr':4, 'Mai':5,'Jun':6,'Jul':7,'Ago':8,'Set':9,'Out':10,'Nov':11,'Dez':12}
dfc = dfc.replace(meses)

In [ ]:
#Cria campo data juntando colunas mês e ano
dfc["data"]= dfc["Ano"].astype(str) + "-" + dfc["Mês"].astype(str)

In [ ]:
#Converte para datetime para poder gerar série
dfc["data"]=pd.to_datetime(dfc.data)


In [ ]:
#Transforma campo data em índice
dfc.set_index('data', inplace=True)

In [ ]:
#Preenchendo missing values com 0,00
dfc.fillna(0, inplace=True)
dfc

In [ ]:
#Soma todos os tributos e armazena em total
dfc['total'] = dfc.sum(axis=1)
dfc

In [ ]:
#Agrupa valores por data, criando df só com data e total
dfc=pd.DataFrame(dfc.groupby(by='data')['total'].sum())
dfc

In [ ]:
#Plotagem da série histórica
plt.figure(figsize=(20,5))
dfc['total'].plot(label = 'Total arrecadado')
plt.ylabel('Valor')
plt.xlabel('Período')
plt.title('Histórico de arrecadação tributária')
plt.legend();

In [ ]:
# 1.356... e+12 significa 1 trilhão e 356 bilhões
dfc.groupby(dfc.index.year).sum()

In [ ]:
#Resumo estatístico
dfc.describe()

In [ ]:
#Analisando tendência mensal
dfc.groupby(dfc.index.month).sum().plot(figsize=(20,5))
plt.ylabel('Valor')
plt.xlabel('Período')
plt.title('Histórico de arrecadação mensal')
plt.xticks(dfc.index.month)#mostrar todos os meses no eixo x
plt.legend();

In [ ]:
dfc.groupby(dfc.index.month).sum()

<H3>SÉRIE HISTÓRICA DE VENDAS DO TESOURO</H3>

In [ ]:
dft

In [ ]:
#identificado que não tem nenhum campo nulo e que data e valor precisam ser convertidos
dft.info()

In [ ]:
#Alterando nomes das colunas para facilitar 
dft.columns = ['tipo', 'dtvecto', 'dtvenda', 'pu','qtde','valor']
dft.info()

In [ ]:
#Alterando a vírgula por ponto para poder converter valor para numérico
v= {',': '.'}
dft = dft.replace(v, regex=True)
dft

In [ ]:
#Convertendo campo valor para numérico
dft["valor"]= pd.to_numeric(dft["valor"])
dft.info()

In [ ]:
#Converte para datetime para poder gerar série
dft["dtvenda"]=pd.to_datetime(dft.dtvenda, format='%d/%m/%Y')

In [ ]:
#Seta data como índice
dft.set_index('dtvenda', inplace=True)

In [ ]:
#pdp.ProfileReport(dfc)

In [ ]:
#Agrupa valores por dia
dft=pd.DataFrame(dft.groupby(by='dtvenda')['valor'].sum())
dft

In [ ]:
#Gráfico com vendas anuais
dft.plot(figsize=(20,8))
plt.ylabel('Valor')
plt.xlabel('Período')
plt.title('Histórico de venda ANUAL')
#plt.legend('Valor');

In [ ]:
# 1.356... e+12 significa 1 trilhão e 356 bilhões
dft.groupby(dft.index.year).sum()

In [ ]:
dft.describe()

In [ ]:
#Analisando tendência mensal
dft.groupby(dft.index.month).sum().plot(figsize=(20,5))
plt.ylabel('Valor')
plt.xlabel('Período')
plt.title('Histórico de venda mensal')
plt.xticks(dft.index.month)
plt.legend();

In [ ]:
dft.groupby(dft.index.month).sum()

In [ ]:
dft.describe()


<H2>SÉRIE HISTÓRICA DO PIB</H2>

In [ ]:
#Bibliotecas para web scraping
import urllib.request
from bs4 import BeautifulSoup

In [ ]:
pib = "http://www.ipeadata.gov.br/ExibeSerie.aspx?serid=521274780&module=M"

In [ ]:

page = urllib.request.urlopen(pib)
soup = BeautifulSoup(page, 'html5lib')

In [ ]:
#Carregando só a classe de interesse
table = soup.find('table', class_='dxgvControl')
#print(table)

In [ ]:
#gerando a lista em colunas
A=[]
B=[]

for row in table.findAll("tr"): #para tudo que estiver em <tr>
    cells = row.findAll('td') #variável para encontrar <td>
    if len(cells)==2: #número de colunas
        A.append(cells[0].find(text=True)) #iterando sobre cada linha
        B.append(cells[1].find(text=True))
        

In [ ]:
#Salvando os dados capturados em um dataframe
dfpib = pd.DataFrame()
dfpib['Data']=A
dfpib['PIB']=B
dfpib

In [ ]:
dfpib.info()

In [ ]:
#Identifica e sumariza valores nulos
dfpib.isnull().sum()

In [ ]:
#Exclui linhas com missing values
dfpib.dropna(inplace=True)

In [ ]:
#Alterando ponto para traço para poder converter para data
dfpib['Data'] = dfpib['Data'].str.replace('.','-')
dfpib

In [ ]:
#Converte campo object para tipo data
dfpib['Data'] = pd.to_datetime(dfpib.Data)
dfpib

In [ ]:
#Seta data como índice
dfpib.set_index('Data', inplace=True)

In [ ]:
#Alterando a vírgula por ponto para poder converter valor para numérico
dfpib['PIB'] = dfpib['PIB'].str.replace('.','')
dfpib

In [ ]:
#Trocando vírgula por ponto para poder converter 
dfpib['PIB'] = dfpib['PIB'].str.replace(',','.')

In [ ]:
#Conversão para numérico
dfpib["PIB"] =pd.to_numeric(dfpib.PIB)
dfpib.info()

In [ ]:
#Multiplicando por 1 milhão pois conforme o site, a tabela estava assim reduzida
dfpib["PIB"] = dfpib["PIB"] * 1000000
dfpib

In [ ]:
#Reduzir o período para os últimos 20 anos
dfpib = dfpib.loc['2001-01-01':'2020-12-01']
dfpib

In [ ]:
#Gráfico com evoluçaõ do PIB anual
dfpib.plot(figsize=(20,8))
plt.ylabel('Valor')
plt.xlabel('Período')
plt.title('Histórico de PIB ANUAL')
plt.legend();

In [ ]:
#Seleciona ano de 2020
dfpib2020 = dfpib.loc['2020-01-01':'2020-12-01']

In [ ]:
#Gráfico com evoluçaõ do PIB anual 2020
dfpib2020.plot(figsize=(20,8))
plt.ylabel('Valor')
plt.xlabel('Mês')
plt.title('Histórico de PIB 2020')
plt.grid(True)
plt.legend();

In [ ]:
#Valor PIB ano a ano
dfpib.groupby(dfpib.index.year).sum()

In [ ]:
dfpib.describe()

In [ ]:
#Analisando tendência mensal
dfpib.groupby(dfpib.index.month).sum().plot(figsize=(20,5))
plt.ylabel('Valor')
plt.xlabel('Mês')
plt.title('Histórico de PIB mensal')
plt.xticks(dfpib.index.month)
plt.legend();

In [ ]:
dfpib.groupby(dfpib.index.month).sum()

<H2> Preparando dados para análise conjunta das 3 séries</h2>


In [ ]:
#Agrupa a série por mês/ano para poder comparar com a arrecadação
dft2= pd.DataFrame(dft.resample('M').valor.sum())
dft2

In [ ]:
#Seleciona mesmo período da arrecadação para comparação
dftc = dft2.loc['2016-01-01':'2020-12-31']
dftc.reset_index(inplace=True) #reseta o índice para dtvenda voltar a ser coluna
dftc['dtvenda'] = pd.to_datetime(dftc["dtvenda"].dt.strftime('%Y-%m')) # converte data para que todos os meses tenham dia = 1
dftc.set_index('dtvenda', inplace=True) #Seta data como índice

In [ ]:
#Seleciona mesmo período da arrecadação para comparação
dfpib = dfpib.loc['2016-01-01':'2020-12-31']

In [ ]:
#Junta a série da arrecadação com vendas do tesouro
dftc = pd.concat([dftc,dfc,dfpib], axis=1) 
dftc.columns = ['TotalTesouro', 'TotalArrecadacao', 'TotalPib'] #altera nome das colunas

In [ ]:
dftc.head()

In [ ]:
#Gráfico normalizado    
(dftc/dftc.iloc[0]*100).plot(figsize=(20,10))
plt.ylabel('Valores normalizados')
plt.xlabel('Período')
plt.title('Comparativo de valores Arrecadação/Vendas tesouro/Pib')
plt.show()

In [ ]:
#Função para criação do mapa de correlação dos impostos
def plot_corr(corr):
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask, 1)] = True
    sns.heatmap(corr, mask=mask, cmap='RdBu', square=True, linewidths=.5)
    
# Cálculo da correlação
corr = dftc.corr() 
plot_corr(corr)

In [ ]:
dftc

## Modelos preditivos

In [ ]:
dft

In [ ]:
#Gravei o dataframe preparado
dft.to_csv('tesouro.csv')

In [ ]:
#Só para teste, lê arquivo, gera index
dft = pd.read_csv('tesouro.csv', sep = ',', dtype=None, low_memory=False, index_col=["dtvenda"])
dft.info()

In [ ]:
#Separando dados de treino e teste
dft_treino = dft.loc['2015-01-01':'2019-12-31']
dft_teste = dft.loc['2020-01-01':'2020-12-31']

In [ ]:
dft_teste.info()

## Modelo auto-arima

In [ ]:
#Carregamento dos dados, teatando novo dataset tesouro
dft_treino_arima = dft_treino
dft_teste_arima = dft_teste

In [ ]:
#Decomposição da série temporal
decomposition = seasonal_decompose(dft_treino_arima, model='multiplicative', period=30)
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid
plt.figure(figsize=(16,10))
plt.subplot(411)
plt.plot(dft_treino_arima)
plt.ylabel('Original')
plt.subplot(412)
plt.plot(trend)
plt.ylabel('Trend')
plt.subplot(413)
plt.plot(seasonal)
plt.ylabel('Seasonality')
plt.subplot(414)
plt.plot(residual)
plt.ylabel('Residuals')
plt.tight_layout()
plt.show()

In [ ]:
#Plotagem da Autocorrelação
#fig, axes = plt.subplots(3, 2, figsize=(12, 18))
#plt.title('Autocorrelação da arrecadação de IR')

# Coordenadas dos eixos para plotagem
#ax_idcs = [     (0, 0),     (0, 1),     (1, 0),     (1, 1),     (2, 0),     (2, 1) ]

#for lag, ax_coords in enumerate(ax_idcs, 1):
 #   ax_row, ax_col = ax_coords
  #  axis = axes[ax_row][ax_col]
   # lag_plot(dfc_treino_arima["total"], lag=lag, ax=axis)
    #axis.set_title(f"Lag={lag}")

#plt.show()

Dickey-Fuller teste: esse é um dos testes estatísticos para verificar Estacionaridade. Aqui, a hipótese nula é que o TS é não-estacionária. Os resultados do teste são compostos por uma estatística de teste e alguns valores críticos para os níveis de confiança da diferença. Se o ‘teste estatístico’ é menor do que o “valor crítico”, podemos rejeitar a hipótese nula e dizer que a série é estacionária

In [ ]:
#Função para verificar a estacionariedade 
#Valor p mostra se série é estacionária ou não, abaixo de 0.05 ´estacionária.
#P é menor que 0.05 e O valor de teste é menor que todos os vlres críticos, portanto é estacionária
def adf_test(y):
    print('Resultado do Teste Dickey-Fuller:')
    dftest = adfuller(y, autolag="AIC")
    dfoutput = pd.Series(dftest[0:4], index=['Teste','Valor p', 'Nº de lags', 'Nº de observações'])
    for key, value in dftest[4].items():
        dfoutput['Valor Crítico ({})'.format(key)] = value
    print(dfoutput)

adf_test(dft_treino_arima)

In [ ]:
#Aplicação da diferenciação
#dfc_treino_arima_diff = dfc_treino_arima - dfc_treino_arima.shift()
#dfc_treino_arima_diff.dropna(inplace=True)

#Plotagem da diferenciação de primeira ordem da PETR4
#plt.figure(figsize=(16,8))
#plt.plot(dfc_treino_arima_diff)
#plt.title('Plotagem da diferenciação da arrecadação')
#p#lt.grid(True)

In [ ]:
#Plotagem dos gráficos ACF e PACF
#Autocorrelação é a relação de um lag(período) com o anterior, quanto mais perto de 1 maior é a correlação. Varia de 1 a -1
#No caso, p lag 0 deu 1 porque a comparação é com ele mesmo. A área mais perto de zero indica menor correlação. 
#A área azul no gráfico representa valores não confiáveis para a análise, somente os que estão fora da área azul são confiáveis
#Na autocorrelação um período influencia no anterior, já na parcial, ela sempre é feita em relação ao período t(0) 

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8,8))
plot_acf(dft_treino_arima, ax = ax1)
plot_pacf(dft_treino_arima, ax = ax2)

#plot_acf(dfc_treino_arima.total, lags=48)

print (ax1)
print (ax2)

plt.show()


In [ ]:
#Definição do modelo
modelo = pm.auto_arima(dft_treino_arima['valor'], start_p=1, start_q=1,
                      max_p=3, max_q=3, # Máximo 'p' e 'q'
                      m=12,              # Frequência da série
                      d=0,           # d do arima, diferenciação
                      stationary = True, #Estacionaridade
                      seasonal=True, #Sazonalidade
                      start_P=0, 
                      D=0, 
                      trace=True, #Se TRUE, a lista de modelos ARIMA considerados será reportada.
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=False) #Se TRUE, fará a seleção gradual (mais rápido). Caso contrário, ele pesquisará todos os modelos.

#Treinamento do modelo
modelo.fit(dft_treino_arima)

In [ ]:
print(modelo.summary())

In [ ]:
n_periods=len(dft_teste_arima['valor'])
#print(n_periods)

In [ ]:
#Faz a predição para 12 meses
forecast_arima = modelo.predict(n_periods=len(dft_teste_arima['valor']))
forecast_arima = pd.DataFrame(forecast_arima, index = dft_teste_arima.index,columns=['Prediction'])
conf_int = modelo.predict(len(dft_teste_arima['valor']), return_conf_int=True, alpha = 0.05) #Pega o intervalo de confiança

In [ ]:
#Plotagem dos dados de treinamento, teste e previsão
plt.figure(figsize=(20,10))
plt.title('Dados de treinamento x Dados de teste x Dados previstos das vendas ')
plt.plot(dft_treino_arima['valor'], color='green', label = 'Dados de treinamento')
plt.plot(dft_teste_arima['valor'], color = 'blue', label = 'Dados de teste')
plt.fill_between(dft_teste_arima.index, conf_int[1][:,0], conf_int[1][:,1], 
                 color='k', alpha=.08)
plt.plot(forecast_arima['Prediction'], color = 'red', label = 'Dados previstos')
plt.xlabel('Período')
plt.ylabel('Preço de fechamento')
plt.legend()
#plt.grid(True)
#plt.savefig('arima1.pdf')
plt.show()

In [ ]:
#Plotagem dos dados de teste e previsão
plt.figure(figsize=(20,8))
plt.title('Dados de teste x Dados previstos da arrecadação')
plt.plot(dft_teste_arima['valor'], color = 'blue', label = 'Dados de teste')
plt.fill_between(dft_teste_arima.index, conf_int[1][:,0], conf_int[1][:,1], 
                 color='k', alpha=.08)
plt.plot(forecast_arima['Prediction'], color = 'red', label = 'Dados previstos')
plt.xlabel('Período')
plt.ylabel('Preço de fechamento')
plt.legend()
#plt.savefig('arima2.pdf')
plt.show()

In [ ]:
#Cálculo do erro
#MSE= erro quadrático médio média do erro das previsões ao quadrado, qto maior pior o modelo
#RMSE = raiz do erro quadrático médio
#MAE = Erro absoluto médio, média das distâncias entre previsão e o real, não pune tanto outliers como os dois anteriores
#MAPE = média do Percentual de erro médio absoluto(módulo dos negativos), quanto menor melhor
mse = mean_squared_error(dft_teste_arima['valor'], forecast_arima['Prediction'])
print('MSE: '+str(mse))
mae = mean_absolute_error(dft_teste_arima['valor'], forecast_arima['Prediction'])
print('MAE: '+str(mae))
rmse = math.sqrt(mean_squared_error(dft_teste_arima['valor'], forecast_arima['Prediction']))
print('RMSE: '+str(rmse))
#mape = mean_absolute_percentage_error(dfc_teste_arima['total'], forecast_arima['Prediction'])
#print('MAPE: '+str(mape))

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mape = mean_absolute_percentage_error(dft_teste_arima['valor'], forecast_arima['Prediction'])
mape

In [ ]:
#nb_predict_train = modelo.predict(dfc_treino_arima)
#print("Exatidão (Accuracy): {0:.4f}".format(metrics.accuracy_score(dfc_teste_arima, forecast_arima['Prediction'])))
#print()

In [ ]:
#Plotagem dos resíduos
residuals = pd.DataFrame(modelo.resid())
residuals.plot()

In [ ]:
#Plotagem da densidade dos resíduos(erros residuais)
residuals.plot(kind='kde')
plt.show()
print(residuals.describe())

<h3>Modelo preditivo com RNN com arquitetura LSTM (Long Short Term Memory)</h3>

In [ ]:
#Carregamento dos dados
dft_treino_lstm = dft_treino
dft_teste_lstm = dft_teste

In [ ]:
dft_treino_lstm

In [ ]:
#Normalização dos dados, entre 0 e 1

train_lstm = dft_treino_lstm.iloc[:,0:1].values# seleciona o valor

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_scaled_lstm = scaler.fit_transform(train_lstm) 


In [ ]:
#Criação de uma estrutura de dados com 60 timesteps e 1 saída 
timesteps = 60
x_train_lstm = []
y_train_lstm = []
for i in range(timesteps, train_lstm.shape[0]):
    x_train_lstm.append(train_scaled_lstm[i-timesteps:i, 0]) #Utilização de 60 timesteps anteriores
    y_train_lstm.append(train_scaled_lstm[i, 0]) 
x_train_lstm, y_train_lstm = np.array(x_train_lstm), np.array(y_train_lstm)

In [ ]:
#Reshaping o dataset de treinamento 
#Sendo o segundo parâmetro como: 
       #x_train_lstm.shape[0] = batch_size, que é o número de vendas do período de treinamento
       #x_train_lstm.shape[1] = time_step, que é o número de vendas anteriores
       
x_train_lstm = np.reshape(x_train_lstm, (x_train_lstm.shape[0], x_train_lstm.shape[1], 1))


In [ ]:
#Criação da LSTM utilizando a biblioteca Keras
#from keras.layers import Bidirectional

# Inicialização da RNN
model_rnn = Sequential()

#O parâmetro return_sequences=True indica que a rede terá mais camadas a frente
#O parâmetro Dropout ajuda no ajuste do Overfitting

# Adiciona a primeira camada LSTM com o Dropout. Units é qtde de neurônio
model_rnn.add((LSTM(units = 128, return_sequences = True, input_shape = (x_train_lstm.shape[1], 1))))
model_rnn.add(Dropout(0.3))

# Adiciona a segunda camada LSTM com o Dropout
model_rnn.add(LSTM(units = 128))
model_rnn.add(Dropout(0.3))

# Adiciona a camada de saída 
#model_rnn.add(Dense(1, activation='linear'))
model_rnn.add(Dense(1, activation='relu'))


# Compila a RNN, neste caso utilizando o otimizador 'Adam', que ajusta hiperparãmetros?
#model_rnn.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=['mean_absolute_error'])
model_rnn.compile(optimizer = 'adam', loss = 'mape', metrics=['msle'])

# Faz o treinamento da RNN utilizando o dataset de treinamento. Épocas são quantas vezes vai iterar

model_rnn.fit(x_train_lstm, y_train_lstm, epochs = 100, batch_size = 32) 

#Mostra o resumo do modelo
model_rnn.summary()

In [ ]:
#Visualização do modelo tive que instalar pydot e graphviz
plot_model(model_rnn, show_shapes=True) #Acrescente o parâmetro 'to_file='model_rnn.png' para exportar a figura

In [ ]:
#Carregamentos dos dados de teste para fazer as predições
test_lstm = dft_teste_lstm.iloc[:,0:1].values 
#test_lstm

In [ ]:
# Concatena os dados de treinamento e teste, pois os preços de fechamento anteriores não estão no conjunto de teste
combine = pd.concat((dft_treino_lstm['valor'], dft_teste_lstm['valor']), axis = 0)

#print(combine)

# Tratamento do conjunto de teste considerando os timesteps anteriores
test_inputs = combine[len(combine) - len(dft_teste_lstm) - timesteps:].values
#print(test_inputs)
test_inputs = test_inputs.reshape(-1,1)

# Normalização dos dados
test_inputs = scaler.transform(test_inputs)

In [ ]:
# Predição utilizando os dados de teste
x_test_lstm = []
for i in range(timesteps, dft_teste_lstm.shape[0]+timesteps):
    x_test_lstm.append(test_inputs[i-timesteps:i, 0])
x_test_lstm = np.array(x_test_lstm)
x_test_lstm = np.reshape(x_test_lstm, (x_test_lstm.shape[0], x_test_lstm.shape[1], 1))
predictions_lstm = model_rnn.predict(x_test_lstm)

# Desnormaliza os dados
predictions_lstm = scaler.inverse_transform(predictions_lstm)

In [ ]:
#Plotagem dos dados de treinamento, teste e previsão
plt.figure(figsize=(16,8))
plt.plot(dft_treino_lstm.index, dft_treino_lstm, color='green', label = 'Dados de treinamento')
plt.plot(dft_teste_lstm.index, test_lstm, color = 'blue', label = 'Dados de teste')
plt.plot(dft_teste_lstm.index, predictions_lstm, color = 'red', label = 'Dados previstos')
plt.title('Dados de treinamento x Dados de teste x Dados previstos da arrecadação')
plt.xlabel('Período')
plt.ylabel('Preço de fechamento')
plt.legend()
#plt.grid(True)
#plt.savefig('lstm1.pdf')
plt.show()

In [ ]:
#Plotagem dos dados de teste e dados previstos
plt.figure(figsize=(16,8))
plt.plot(dft_teste_lstm.index, test_lstm, color = 'blue', label = 'Dados de teste')
plt.plot(dft_teste_lstm.index, predictions_lstm, color = 'red', label = 'Dados previstos')
plt.title('Dados previstos x Dados de teste das vendas')
plt.xlabel('Período')
plt.ylabel('Preço de fechamento')
plt.legend()
#plt.grid(True)
#plt.savefig('lstm2.pdf')
plt.show()

In [ ]:
#Cálculo do erro
print('MAE: ', mean_absolute_error(test_lstm,predictions_lstm))
print('MSE: ', mean_squared_error(test_lstm,predictions_lstm))
print('RMSE: ', np.sqrt(mean_squared_cerror(test_lstm,predictions_lstm)))

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mape = mean_absolute_percentage_error(test_lstm, predictions_lstm)
mape

In [ ]:
print('Nº de (Treino e teste):' + str((len(dft_treino_lstm), len(dft_teste_lstm))))

### FACEBOOK 

In [ ]:
#Carregamento dos dados de treinamento e teste
dft_treino_prophet = dft_treino
dft_teste_prophet = dft_teste

In [ ]:
dft_treino.info()

In [ ]:
dft_treino_prophet.info()

In [ ]:
dft_teste_prophet.info()

In [ ]:
#Renomeia as colunas 'total' para 'y' e 'data' para 'ds' do conjunto de treinamento
dft_treino_prophet = pd.DataFrame({"ds": dft_treino_prophet.index, "y": dft_treino_prophet.valor})
dft_treino_prophet.reset_index(drop = True, inplace = True)
dft_treino_prophet.info()

In [ ]:
#Renomeia as colunas 'valor' para 'y' e 'data' para 'ds' do conjunto de treinamento
dft_teste_prophet = pd.DataFrame({"ds_test": dft_teste_prophet.index, "y_test": dft_teste_prophet.valor})
dft_teste_prophet.reset_index(drop = True, inplace = True)
dft_teste_prophet.info()

In [ ]:
#Utiliza a biblioteca Prophet para fazer a previsão
#parâmetro changepoint_prior_scale ajusta a tendência
#parâmetro seasonality_prior ajusta sazonaliadde
#interval_width pode ser de 80% ou 95%
from fbprophet import Prophet

prophet_model = Prophet(changepoint_prior_scale=0.15, interval_width=0.95, seasonality_prior_scale=0.01, 
                        daily_seasonality=True, seasonality_mode='multiplicative')
prophet_model.fit(dft_treino_prophet)


In [ ]:
#Cria datas futuras e faz a predição
prophet_forecast = prophet_model.make_future_dataframe(periods=365, freq='D')
prophet_forecast = prophet_model.predict(prophet_forecast)

#Plotagem do gráfico de previsão
fig = prophet_model.plot(prophet_forecast)
ax1 = fig.gca()
ax1.set_title("Previsão de vendas", fontsize=16)
ax1.set_xlabel("Período", fontsize=12)
ax1.set_ylabel("Valor", fontsize=12)

In [ ]:
#yhat é o valor predito, yhat_lower e uppersão os valres para intervalo de certeza?
#prophet_forecast
prophet_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()


In [ ]:
#Seleciona os dados previstos apenas para o período de teste
train_end_date = "2019-12-31"
prophet_forecast = prophet_forecast[prophet_forecast['ds'] > train_end_date]
prophet_forecast.info()

In [ ]:
#Plotagem dos componententes da previsão
fig = prophet_model.plot_components(prophet_forecast)

In [ ]:
#Exclusão dos fins de semana nos dados previstos
prophet_forecast = prophet_forecast[prophet_forecast['ds'].dt.dayofweek < 5]

#Exclusão dos feriados nos dados previstos
holidays = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(['2020-01-01','2020-02-24','2020-02-25','2020-04-10','2020-04-21','2020-05-01','2020-06-11','2020-09-07','2020-10-12','2020-11-02','2020-11-15','2020-12-25','2020-12-31']),
})

prophet_forecast = prophet_forecast[~prophet_forecast['ds'].isin(holidays['ds'])]
prophet_forecast = prophet_forecast.dropna()


In [ ]:
#Converte para data
prophet_forecast['ds']= pd.to_datetime(prophet_forecast['ds'])
dft_teste_prophet['ds_test']= pd.to_datetime(dft_teste_prophet['ds_test'])
dft_treino_prophet['ds']= pd.to_datetime(dft_treino_prophet['ds'])

In [ ]:
#Escolhe as datas ('ds' e 'ds_test') como índices
prophet_forecast.set_index(prophet_forecast['ds'], inplace=True) 
dft_teste_prophet.set_index(dft_teste_prophet['ds_test'], inplace=True) 
dft_treino_prophet.set_index(dft_treino_prophet['ds'], inplace=True) 

In [ ]:
#Plotagem do comparativo entre o preço previsto e o dataset
n = dft_treino_prophet.shape[0]
plt.figure(figsize=(14,8))
plt.title('Dados de treinamento x Dados de teste x Dados previstos das vendas')
plt.plot(dft_treino_prophet['y'], 'green', label='Dados de treinamento')
plt.plot(dft_teste_prophet['y_test'], color = 'blue', label='Dados de teste')
plt.plot(prophet_forecast['yhat'][-n:], color = 'red', label = 'Dados previstos')
plt.fill_between(prophet_forecast.index[-n:], prophet_forecast['yhat_lower'][-n:], prophet_forecast['yhat_upper'][-n:], color='k', alpha=.08, label="Banda de variação")
plt.xlabel("Período")
plt.ylabel("Preço de fechamento")
plt.legend()
plt.savefig('prophet1.pdf')
plt.grid(True)

#Plotagem do comparativo entre o preço previsto e o preço de validação
plt.figure(figsize=(14,8))
plt.plot(dft_teste_prophet['y_test'], color = 'blue', label='Dados de teste')
plt.plot(prophet_forecast['yhat'], color = 'red', label = 'Dados previstos')
plt.fill_between(prophet_forecast.index[-n:], prophet_forecast['yhat_lower'][-n:], prophet_forecast['yhat_upper'][-n:], color='k', alpha=.08, label="Banda de variação")
plt.xlabel("Período")
plt.ylabel("Preço de fechamento")
plt.legend()
plt.grid(True)
#plt.savefig('prophet2.pdf')
#plt.show()

In [ ]:
#Cálculo do erro
print('MAE: ', mean_absolute_error(dft_teste_prophet['y_test'],prophet_forecast['yhat']))
print('MSE: ', mean_squared_error(dft_teste_prophet['y_test'],prophet_forecast['yhat']))
print('RMSE: ', np.sqrt(mean_squared_error(dft_teste_prophet['y_test'],prophet_forecast['yhat'])))

In [ ]:
mape = mean_absolute_percentage_error(dft_teste_prophet['y_test'], prophet_forecast['yhat'])
mape

<h3>Modelo LSTM treinado utilizado nos dados futuros</h3>
Este foi o modelo escolhido por ter o menor MAPE

In [ ]:
#Só para teste, lê arquivo, gera index
dft = pd.read_csv('tesouro.csv', sep = ',', dtype=None, low_memory=False, index_col=["dtvenda"])
futuro_teste_dft_lstm =  dft.loc['2021-01-01':'2021-03-31']

In [ ]:
futuro_teste_dft_lstm.info()

In [ ]:
# Concatena os dados de treinamento e teste, pois os preços de fechamento anteriores não estão no conjunto de teste
combine = pd.concat((dft_teste_lstm[-timesteps:]['valor'], futuro_teste_dft_lstm['valor']), axis = 0)

# Tratamento do conjunto de teste considerando os timesteps anteriores
teste_inputs = combine[len(combine) - len(futuro_teste_dft_lstm['valor']) - timesteps:].values
teste_inputs = teste_inputs.reshape(-1,1)

# Normalização dos dados
teste_inputs = scaler.transform(teste_inputs)

In [ ]:
# Predição utilizando os dados de teste futuro
x_test_lstm = []
for i in range(timesteps, futuro_teste_dft_lstm.shape[0]+timesteps):
    x_test_lstm.append(teste_inputs[i-timesteps:i, 0])
x_test_lstm = np.array(x_test_lstm)
x_test_lstm = np.reshape(x_test_lstm, (x_test_lstm.shape[0], x_test_lstm.shape[1], 1))
predictions_lstm = model_rnn.predict(x_test_lstm)

# Desnormaliza os dados
predictions_lstm = scaler.inverse_transform(predictions_lstm)

In [ ]:
#Plotagem dos dados da janela, dados de teste futuro e dados previstos
plt.figure(figsize=(16,8))
plt.plot(combine.index, combine.values, color = 'green', label = 'Dados da janela')
plt.plot(futuro_teste_dft_lstm.index, futuro_teste_dft_lstm, color = 'blue', label = 'Dados de teste futuro')
plt.plot(futuro_teste_dft_lstm.index, predictions_lstm, color = 'red', label = 'Dados previstos')
plt.title('Dados previstos x Dados de teste futuro da PETR4')
plt.xlabel('Período')
plt.ylabel('Preço de fechamento')
#plt.xticks(futuro_teste_dft_lstm.index.year())
plt.legend()
plt.grid(True)
#plt.savefig('lstm3.pdf')
plt.show()

In [ ]:
#Plotagem dos dados de teste e dados previstos
plt.figure(figsize=(16,8))
plt.plot(futuro_teste_dft_lstm.index, futuro_teste_dft_lstm, color = 'blue', label = 'Dados de teste futuro')
plt.plot(futuro_teste_dft_lstm.index, predictions_lstm, color = 'red', label = 'Dados previstos')
plt.title('Dados previstos x Dados de teste futuro da PETR4')
plt.xlabel('Período')
plt.ylabel('Preço de fechamento')
plt.legend()
plt.grid(True)
#plt.savefig('lstm4.pdf')
plt.show()

In [ ]:
#Cálculo do erro
print('MAE: ', mean_absolute_error(futuro_teste_dft_lstm,predictions_lstm))
print('MSE: ', mean_squared_error(futuro_teste_dft_lstm,predictions_lstm))
print('RMSE: ', np.sqrt(mean_squared_error(futuro_teste_dft_lstm,predictions_lstm)))

In [ ]:
#Calcula percentual de erro médio absoluto
mape = mean_absolute_percentage_error(futuro_teste_dft_lstm, predictions_lstm)
mape

**FIM**